In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

from sklearn.linear_model import LinearRegression #For regression model
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2023/train_data.csv
/kaggle/input/widsdatathon2023/test_data.csv
/kaggle/input/widsdatathon2023/sample_solution.csv


In [2]:
def seperate_monthly(df):
    df[['month','day','year']]=df.startdate.str.split('/',expand=True)
    df=df.drop('day', axis=1)
    df=df.drop('startdate',axis=1)
    return df

In [3]:
def seperate_y(df):
    y_df=df['contest-tmp2m-14d__tmp2m']
    X_df=df.drop('contest-tmp2m-14d__tmp2m',axis=1)
    return y_df,X_df

In [4]:
def normanlize_data(df,train_df):
    for column in df.columns:
        df[column] = df[column]  /  train_df[column].abs().max()
    return df

In [5]:
train_df=pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv')
train_df=train_df.dropna(axis=0)
test_df=pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv')
test_df=test_df.dropna(axis=0)
train_df

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
0,0,0.0,0.833333,9/1/14,237.00,29.02,31.64,29.57,30.73,29.71,...,-27.68,-37.21,8.32,9.56,-2.03,48.13,28.09,-13.50,11.90,4.58
1,1,0.0,0.833333,9/2/14,228.90,29.02,31.64,29.57,30.73,29.71,...,-21.13,-36.57,8.77,21.17,4.44,48.60,27.41,-23.77,15.44,3.42
2,2,0.0,0.833333,9/3/14,220.69,29.02,31.64,29.57,30.73,29.71,...,-10.72,-34.16,6.99,32.16,5.01,48.53,19.21,-33.16,15.11,4.82
3,3,0.0,0.833333,9/4/14,225.28,29.02,31.64,29.57,30.73,29.71,...,0.33,-31.04,6.17,39.66,-1.41,50.59,8.29,-37.22,18.24,9.74
4,4,0.0,0.833333,9/5/14,237.24,29.02,31.64,29.57,30.73,29.71,...,9.83,-31.80,7.47,38.62,-5.21,54.73,-2.58,-42.30,21.91,10.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375729,375729,1.0,0.866667,8/27/16,312.05,23.13,27.20,20.25,24.43,18.35,...,-15.64,-75.68,-3.09,6.93,-16.69,16.98,-13.85,50.25,-31.33,0.77
375730,375730,1.0,0.866667,8/28/16,305.82,23.13,27.20,20.25,24.43,18.35,...,-7.59,-76.42,-13.55,13.36,-15.96,20.45,-16.36,51.65,-30.73,10.10
375731,375731,1.0,0.866667,8/29/16,311.62,23.13,27.20,20.25,24.43,18.35,...,-6.25,-70.65,-23.93,22.62,-16.71,20.28,-15.48,48.58,-18.74,9.28
375732,375732,1.0,0.866667,8/30/16,304.54,23.13,27.20,20.25,24.43,18.35,...,-7.16,-57.67,-33.55,32.06,-16.07,16.60,-20.61,39.23,-16.26,-0.22


In [6]:
#Shuffeling the data
train_df = train_df.sample(frac = 1)

In [7]:
le = preprocessing.LabelEncoder()

# Converting string labels into numbers.
train_df['climateregions__climateregion']=le.fit_transform(train_df['climateregions__climateregion'])
test_df['climateregions__climateregion']=le.fit_transform(test_df['climateregions__climateregion'])

In [8]:
print('training set prep')
train_df=seperate_monthly(train_df)
print('test set prep')
test_df=seperate_monthly(test_df)

training set prep
test set prep


In [9]:
train_df=train_df.astype('float')
test_df=test_df.astype('float')

In [10]:
y_train_df=train_df['contest-tmp2m-14d__tmp2m']
X_train_df=train_df.drop('contest-tmp2m-14d__tmp2m', axis=1)
type(y_train_df)
y_train_df=y_train_df.to_frame()
type(y_train_df)

pandas.core.frame.DataFrame

In [11]:
X_train_df=normanlize_data(X_train_df,train_df)
test_df=normanlize_data(test_df,train_df)

In [12]:
Regresion_model= model = LinearRegression() #Define model object

In [13]:
X_train_df

,index,lat,lon,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,...,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20,month,year
292316,0.777989,0.818182,0.666667,0.455448,0.364468,0.456286,0.438155,0.396094,0.365315,0.431551,...,-0.149201,0.240301,-0.074682,0.048737,-0.212492,0.517060,0.431135,0.333747,0.500000,1.0000
321347,0.855254,0.909091,0.166667,0.018487,0.141353,0.237580,0.205451,0.285340,0.261818,0.281356,...,0.144256,0.575850,-0.344173,0.091063,0.675995,-0.353899,-0.030693,0.571058,0.916667,0.9375
297133,0.790809,0.818182,0.900000,0.350963,0.731707,0.829864,0.712788,0.757505,0.580979,0.662060,...,-0.333492,0.160527,0.087013,0.034031,-0.605767,0.810031,-0.361816,-0.292240,0.666667,0.9375
25808,0.068687,0.227273,0.433333,0.376006,0.300721,0.374410,0.388140,0.354707,0.369790,0.327510,...,0.075599,0.740737,-0.436158,-0.000471,0.057672,0.307978,0.180908,0.388563,0.333333,0.9375
51058,0.135889,0.272727,0.800000,0.294691,0.482539,0.543158,0.499850,0.561061,0.490070,0.462581,...,0.276531,-0.087075,-0.360183,-0.454751,0.144605,0.815290,0.169593,-0.191857,0.416667,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266940,0.710451,0.772727,0.466667,0.029583,-0.166851,-0.071885,-0.277029,-0.215681,-0.227133,-0.262842,...,-0.028718,0.415214,-0.092084,-0.556184,-0.176101,0.135839,0.356483,-0.225806,0.083333,0.9375
317481,0.844964,0.909091,0.000000,0.160462,0.113359,0.151540,0.244684,0.224424,0.189650,0.233116,...,0.147109,0.318330,-0.233890,0.078431,-0.054905,0.281042,0.412407,0.472254,0.333333,0.9375
83345,0.221820,0.363636,0.633333,0.116872,0.681818,0.710519,0.784666,0.641212,0.576503,0.605215,...,0.725181,0.183849,-0.112172,0.596437,-0.256863,-0.350180,0.488880,0.010828,0.750000,0.8750
174938,0.465591,0.590909,0.066667,0.252822,0.122228,0.154593,0.226415,0.194404,0.183776,0.267275,...,0.108311,0.179381,-0.118337,0.091252,-0.017983,0.186378,0.385226,0.411121,0.333333,0.9375


In [14]:
model.fit(X_train_df, y_train_df)

LinearRegression()

In [15]:
predictions=model.predict(test_df)

In [16]:
final=pd.DataFrame(predictions,columns=['contest-tmp2m-14d__tmp2m'])

In [17]:
i=375734
final['index']=0
while i<407088:
    final['index'][i-375734]=i
    i+=1
final

,contest-tmp2m-14d__tmp2m,index
0,42.403806,375734
1,42.245088,375735
2,42.045613,375736
3,41.965447,375737
4,41.957965,375738
...,...,...
31349,11.041618,407083
31350,10.996495,407084
31351,9.925655,407085
31352,10.333649,407086


In [18]:
final.to_csv('/kaggle/working/regression_wetherprediction.csv',index=False)